<a href="https://colab.research.google.com/github/Dada-Tech/multimodal-video-trimming/blob/main/MultiModal_Video_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Configuration

In [14]:
import os
import subprocess

notebook_mode = "auto"
colab_mode = True # for special requirements.txt without numpy/pandas

# Check for Notebook mode
notebook_mode = (True if 'COLAB_GPU' in os.environ else False) if notebook_mode == "auto" else notebook_mode

print(f"""Notebook Mode: {notebook_mode}""")

Notebook Mode: True


In [15]:
def print_section(message):
  """Prints a section separator with a custom message embedded.

  Args:
    message: The message to embed within the separator.
  """
  separator_length = 40
  separator_char = "="

  # Calculate padding for the message
  message_length = len(message)
  padding = (separator_length - message_length - 2) // 2
  padding = max(padding, 0)

  # Separator line
  top_line = separator_char * separator_length

  # Message line
  message_line = separator_char * padding + " " + message + " " + separator_char * padding
  if len(message_line) == 39:
    message_line += separator_char
  message_line = message_line[:separator_length]

  print(top_line)
  print(message_line)
  print(top_line,'\n')


def print_info(message, preview = None, max_length=80):
  print(f"""\n=== {message}\n""")
  if preview:
    print("--- Preview:")
    print(preview[:max_length] + "..." if len(preview) > max_length else preview)


def notebook_mode_print(message_or_df):
  if notebook_mode:
    display(message_or_df) if isinstance(message_or_df, pd.DataFrame) else print(message_or_df)

def ts_to_s(timestamp):
    """Converts a timestamp string in HH:MM:SS.mmm format to seconds.

    Args:
        timestamp: The timestamp string in HH:MM:SS.mmm format.

    Returns:
        The timestamp in seconds as a float.
    """
    hours, minutes, seconds_milliseconds = re.split(r':', timestamp)
    seconds, milliseconds = seconds_milliseconds.split('.')

    # Convert to seconds
    total_seconds = int(hours) * 3600 + int(minutes) * 60 + int(seconds) + int(milliseconds) / 1000.0

    return total_seconds


def generate_experiment_filename(id_token, base, filename_without_extension, extension):
  m1w = hyperparameters["metric_1"]["weight"]
  m2w = hyperparameters["metric_2"]["weight"]
  mdt = hyperparameters["deletion_metric"]["threshold"]

  experiment_name = f"_m1w-{m1w}_m2w-{m2w}_mdt-{mdt}_{id_token}"
  return os.path.join(base, filename_without_extension + experiment_name + "." + extension)


# Define the function to count words while removing punctuation
def count_words_without_punctuation(sentence):
    words = word_tokenize(sentence)

    # Filter out punctuation
    words = [word for word in words if word not in string.punctuation]
    return len(words)

## Installation & Setup

In [3]:
if notebook_mode:
    print_section("installing deps")

    requirements_txt = "requirements-colab.txt" if colab_mode else "requirements.txt"

    # download requirements.txt from repository
    subprocess.run(["curl", "-O", "https://raw.githubusercontent.com/Dada-Tech/multimodal-video-trimming/main/" + requirements_txt], check=True)

    subprocess.check_call(['python', '-m', 'pip', 'install', '--no-cache-dir', '-r', requirements_txt])

    print_info("installation done.")
else:
    print_info("skipping installation")

=========== installing deps ============


=== installation done.



In [4]:
from pydantic import BaseModel, validator, conint, confloat, ValidationError
from enum import Enum
import argparse

class AutoSummary(BaseModel):
    summary_length_percentage: confloat(ge=0.2, le=0.5)
    min_summary_length: conint(ge=30, le=60)
    max_summary_length: conint(ge=100, le=3000)

class DeletionMetric(BaseModel):
    threshold: confloat(ge=0.05, le=0.5)
    smoothing: confloat(ge=0.0, le=1.0)

class Metric1(BaseModel):
    model_size: str
    weight: confloat(ge=0, le=1.0)
    min_words: conint(ge=50, le=1000)

class Metric2(BaseModel):
    weight: confloat(ge=0, le=1.0)
    min_scene_len: conint(ge=15, le=9000)
    threshold: conint(ge=10, le=50)

class Hyperparameters(BaseModel):
    auto_summary: AutoSummary
    deletion_metric: DeletionMetric
    metric_1: Metric1
    metric_2: Metric2  # Add Metric2 with constraints

## Inputs & Hyperparameters

### Auto Summary

*   **`summary_length_percentage`**: 0.3
    *   Determines the target length of the summary as a percentage of the original text length.
*   **`min_summary_length`**: 30
    *   Sets the minimum number of words (or tokens) allowed in the summary.
*   **`max_summary_length`**: 600
    *   Sets the maximum number of words (or tokens) allowed in the summary.

### Deletion Metric

*   **`threshold`**: 0.3
    *   Deleting the bottom **`n`** percent of scores.
*   **`smoothing`**: Smooths the silence between sentneces so they dont get a value of 0. between `0` and `1.0`

### Metric 1
*   **`weight`**: Values between `0` and `1.0`
    *   Controls the influence of this metric on the final score
*   **`model_size`**: either `base` or `large`
    *   Controls which model used for text tasks for this metric.
*   **`min_words`**: minimum number of words for this
 metric to be used. Prevents this text-based metric from being used on cinematic type videos.

### Metric 2

*   **`weight`**: Values between `0` and `1.0`
    *   Controls the influence of this metric on the final score
*   **`min_scene_len`**: Values between `15` and `9000`
    *   Defines the minimum number of frames required for a scene to be considered.
    *   Ensures scenes are at least half a second long (15 frames) and at most 5 minutes (9000 frames at 30fps).
*   **`threshold`**: Values between `10` and `50`
    *   Sets the sensitivity for scene detection.
    *   Lower values detect more cuts, while higher values make detection stricter.

In [55]:
if notebook_mode:
  # video_input = "dataset/teamwork in the classroom.mov"
  # video_output = "dataset/teamwork in the classroom_skimmed.mov"

  video_input = "dataset/z_6gVvQb2d0.mp4"
  video_output = "dataset/z_6gVvQb2d0_skimmed.mp4"

  experiment_mode = False ## If experiment mode is True, you must run SRT File Gen block
  skip_nlp_downloads = False

  export_original_text = False
  export_trimmed_text = False
  export_summarized_text = False

  video_export_max_length_seconds = 0 # set develop video max length to export a shortened version of the multimedia

  # original was max_length=150, min_length=30
  hyperparameters = {
      "auto_summary": {
        "summary_length_percentage": 0.3,
        "min_summary_length": 30,
        "max_summary_length": 600
      },
      "deletion_metric": {
          "threshold": 0.2,
          "smoothing": 0.8
      },
      "metric_1": {
          "model_size": "base",
          "weight": 1,
          "min_words": 50
      },
      "metric_2": {
          "weight": 0.3,
          "min_scene_len": 15,
          "threshold": 25
      }
  }

else:
  # Define the argparse parser
  parser = argparse.ArgumentParser(description="Process video and hyperparameters.")

  # Define the arguments for the inputs
  parser.add_argument("--video_input", "-i", type=str, required=True, help="Path to the video input file")
  parser.add_argument("--video_output", "-o", type=str, default=None, help="Path to save the output video")
  parser.add_argument("--experiment_mode", "-exp", action="store_true", help="Run the project in experiment mode. (No video exports, just timestamps)")
  parser.add_argument("--video_export_max_length_seconds", type=int, default=0, help="Maximum length of the video to export (in seconds)")
  parser.add_argument("--export_original_text", action="store_true", help="Export original text (from video)")
  parser.add_argument("--export_trimmed_text", action="store_true", help="Export trimmed text")
  parser.add_argument("--export_summarized_text", action="store_true", help="Export summarized text")
  parser.add_argument("--skip_nlp_downloads", action="store_true", help="skip nltk library downloads")


  # Hyperparameters as individual arguments
  parser.add_argument("--auto_summary_summary_length_percentage", type=float, default=0.3, help="Summary length as a percentage")
  parser.add_argument("--auto_summary_min_summary_length", type=int, default=30, help="Minimum summary length")
  parser.add_argument("--auto_summary_max_summary_length", type=int, default=600, help="Maximum summary length")

  parser.add_argument("--deletion_metric_threshold", type=float, default=0.2, help="Threshold for deletion metric")
  parser.add_argument("--deletion_metric_smoothing", type=float, default=0.8, help="Smoothing factor for deletion metric")

  parser.add_argument("--metric_1_model_size", type=str, choices=["base", "large"], default="base", help="Model size for metric 1")
  parser.add_argument("--metric_1_weight", type=float, default=1.0, help="Weight for metric 1 (contribution to final score)")
  parser.add_argument("--metric_1_min_words", type=int, default=50, help="Minimum number of words to determine this metric applicable")

  parser.add_argument("--metric_2_weight", type=float, default=0.3, help="Weight for metric 2 (contribution to final score)")
  parser.add_argument("--metric_2_min_scene_len", type=int, default=15, help="Minimum scene length for metric 2")
  parser.add_argument("--metric_2_threshold", type=int, default=25, help="Threshold for scene detection for metric 2")

  # Parse arguments
  args = parser.parse_args()

  # Now you can use the parsed arguments
  video_input = args.video_input
  experiment_mode = args.experiment_mode
  skip_nlp_downloads = args.skip_nlp_downloads
  video_export_max_length_seconds = args.video_export_max_length_seconds
  video_output = args.video_output
  export_original_text = args.export_original_text
  export_trimmed_text = args.export_trimmed_text
  export_summarized_text = args.export_summarized_text

  hyperparameters = {
      "auto_summary": {
          "summary_length_percentage": args.auto_summary_summary_length_percentage,
          "min_summary_length": args.auto_summary_min_summary_length,
          "max_summary_length": args.auto_summary_max_summary_length
      },
      "deletion_metric": {
          "threshold": args.deletion_metric_threshold,
          "smoothing": args.deletion_metric_smoothing
      },
      "metric_1": {
          "model_size": args.metric_1_model_size,
          "weight": args.metric_1_weight,
          "min_words": args.metric_1_min_words
      },
      "metric_2": {
          "weight": args.metric_2_weight,
          "min_scene_len": args.metric_2_min_scene_len,
          "threshold": args.metric_2_threshold
      }
  }

if experiment_mode:
  print_section("Experiment Mode")

# Validate Hyperparameters
try:
    validated_hyperparameters = Hyperparameters(**hyperparameters)
except ValidationError as e:
    print(f"Hyperparameter validation error: {e}")

    print_info("exiting...")
    os._exit(1)

## Imports



In [6]:
print_info("importing...")

import os
import numpy as np
import pandas as pd
import tarfile
import gdown
import re
from functools import reduce
import subprocess
import json

# ML General
from datasets import load_dataset
import torch
import torchaudio
import torch.nn.functional as F
from transformers import LEDTokenizer, LEDForConditionalGeneration
from sentence_transformers import SentenceTransformer

# Text
import string
import pytextrank
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import spacy
import srt

# Audio
import whisperx
import silero_vad
from silero_vad import load_silero_vad, read_audio, get_speech_timestamps
from pydub import AudioSegment

# Video
import ffmpeg
from scenedetect import detect, ContentDetector

print_info("importing done")


=== importing...

/usr/local/lib/python3.11/dist-packages


DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover



=== importing done



In [7]:
if not skip_nlp_downloads:
  print_info("downloading NLTK libraries...")

  nltk.download('punkt')
  nltk.download('punkt_tab')

  # Load the spaCy model
  spacy.cli.download("en_core_web_sm")
  sp = spacy.load('en_core_web_sm')

  print_info("downloading done")


=== downloading NLTK libraries...



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.

=== downloading done



## Variables

In [56]:
full_base = os.path.dirname(video_input)
path_dataset = full_base
filename = os.path.basename(video_input)
filename_without_extension = os.path.splitext(filename)[0]
filename_video_extension = video = os.path.splitext(video_input)[1]

# Video Input
filename_video_input = filename

# Output Filenames
filename_subtitles_output = filename_without_extension + ".srt"
filename_audio_output = filename_without_extension + ".wav"
filename_audio_output_skimmed = filename_without_extension + "_skimmed.wav"

# Text Output Filenmes
filename_paragraph_original = os.path.join(full_base, filename_without_extension + "_paragraph_original.txt")
filename_paragraph_trimmed = os.path.join(full_base, filename_without_extension + "_paragraph_trimmed.txt")
filename_paragraph_summarized = os.path.join(full_base, filename_without_extension + "_paragraph_summarized.txt")

# Experiment CSV
filename_experiment_sentences = generate_experiment_filename("sentences", full_base, filename_without_extension, "csv")
filename_experiment_hyperparameters = generate_experiment_filename("hyperparameters", full_base, filename_without_extension, "json")

# Output
subtitles_output = os.path.join(full_base, filename_subtitles_output)
audio_output = os.path.join(full_base, filename_audio_output)
audio_output_skimmed = os.path.join(full_base, filename_audio_output_skimmed)

if video_output:
  filename_video_output_skimmed = os.path.basename(video_output)
  video_output_skimmed = video_output
else:
  filename_video_output_skimmed = filename_without_extension + "_skimmed" + filename_video_extension
  video_output_skimmed = os.path.join(full_base, filename_video_output_skimmed)

video = ''
audio = ''
subtitles = ''
sentences = ''

## Functions

In [9]:
def drop_if_exists(df, col_name):
  """Drops a column from a DataFrame if it exists
  Args:
    df: The pandas DataFrame to modify.
    col_name: The name of the column to drop and insert.
  """
  if col_name in df.columns:
    df.drop(col_name, axis=1, inplace=True)

In [10]:
def paragraph_to_file(text, filename):
    try:
        # Process the text with spaCy to split into sentences
        doc = sp(text)

        sentences = [sent.text.strip() for sent in doc.sents]

        # Write each sentence to the file
        with open(filename, 'w') as file:
            for sentence in sentences:
                file.write(sentence + "\n")

        print(f"Export Successful: {filename}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [11]:
def get_video_length(input_video):
    """Get the duration (length) of a video file using ffmpeg-python."""
    probe = ffmpeg.probe(input_video, v='error', select_streams='v:0', show_entries='format=duration')
    return float(probe['format']['duration'])

In [12]:
def seconds_to_srt_timestamp(seconds):
    """
    Extract hours, minutes, seconds, and milliseconds
    from a given number of seconds.
    """

    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = seconds % 60
    milliseconds = int((seconds - int(seconds)) * 1000)

    # Format as HH:MM:SS,MS
    return f"{hours:02}:{minutes:02}:{int(seconds):02},{milliseconds:03}"

## Datasets

- teamwork in the classroom.mov - `190MB`
- flipped learning basics.mov - `380MB`
- assessing students without exams.mov - `830MB`

In [54]:
if notebook_mode:
  from google.colab import files

  # Google Drive Dataset Location
  folder_id = '1k7DLJPl1xz9lpU4l3dZYtPe1XawhrXeC' # taken from drive.google.com/drive/u/1/folders/1k7D...(this part)
  gdown.download_folder(id=folder_id, quiet=False, use_cookies=False)

Retrieving folder contents


Processing file 1wslcvTNd88FQMXJgvGbKR3sjwXORR6xt teamwork in the classroom.mov
Processing file 1ZsA-X-HsSk0WJiZaGcHIC3t17DchdXIX teamwork in the classroom.srt
Processing file 1lb2rCvjouElVhqEHAyojUOvb_ECgUqjF teamwork in the classroom.wav
Processing file 1M9BtN3vT0SEo5Os5NSR2APDznFvxFcga uGu_10sucQo.mp4
Processing file 1sETcDGcrTCUCAkn1KwwXYlFpGGe0c8_m z_6gVvQb2d0.mp4


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1wslcvTNd88FQMXJgvGbKR3sjwXORR6xt
From (redirected): https://drive.google.com/uc?id=1wslcvTNd88FQMXJgvGbKR3sjwXORR6xt&confirm=t&uuid=50fc94fe-f72e-4b67-80fa-861f4fdcbd7c
To: /content/dataset/teamwork in the classroom.mov
100%|██████████| 202M/202M [00:01<00:00, 115MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZsA-X-HsSk0WJiZaGcHIC3t17DchdXIX
To: /content/dataset/teamwork in the classroom.srt
100%|██████████| 7.73k/7.73k [00:00<00:00, 17.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1lb2rCvjouElVhqEHAyojUOvb_ECgUqjF
To: /content/dataset/teamwork in the classroom.wav
100%|██████████| 62.4M/62.4M [00:00<00:00, 216MB/s]
Downloading...
From: https://drive.google.com/uc?id=1M9BtN3vT0SEo5Os5NSR2APDznFvxFcga
To: /content/dataset/uGu_10sucQo.mp4
100%|██████████| 8.26M/8.26M [00:00<00:00, 144MB/s]
Downloa

# Preprocessing - Audio

In [57]:
print_section("Preprocessing")

============ Preprocessing =============



## Audio - Extract

In [58]:
# Extract audio (wav) from video
# !ffmpeg -y -i "$video_input" -vn -acodec pcm_s16le -ar 44100 -ac 2 "$audio_output"
print_info("extracting audio from video")

subprocess.run(['ffmpeg', '-y', '-i', video_input, '-vn', '-acodec', 'pcm_s16le', '-ar', '44100', '-ac', '2', audio_output], check=True)
# subprocess.run(["ffmpeg", '-y', '-i', video_input, '-vn', '-acodec', 'pcm_s16le', '-ar', '44100', '-ac', '2', audio_output], check=True, capture_output=True)


=== extracting audio from video



CompletedProcess(args=['ffmpeg', '-y', '-i', 'dataset/z_6gVvQb2d0.mp4', '-vn', '-acodec', 'pcm_s16le', '-ar', '44100', '-ac', '2', 'dataset/z_6gVvQb2d0.wav'], returncode=0)

## Audio - SRT File Generation

##### Time Taken: ~4min

SRT  
each **`subtitle`** in the subtitles array has the following properties:

1. **`index`**
   - The sequential number of the subtitle within the SRT file.
   - `1`, `2`, `3`, etc. (Integer)
2. **`start`**
   - The time (in milliseconds) when the subtitle should appear on the screen.
   - `00:00:05,000` (String representing HH:MM:SS,SSS)
3. **`end`**
   - The time (in milliseconds) when the subtitle should disappear from the screen.
   - `00:00:10,000` (String representing HH:MM:SS,SSS)
4. **`content`**
   - The actual text of the subtitle that will be displayed.
   - "Hello, world!" (String)
5. **`proprietary`**
   - This field holds any additional data or formatting specific to the SRT file or software used to create it. Often empty and can usually be ignored.
   - `''` (Empty string, or sometimes contains specific formatting codes)

In [ ]:
# Select device (GPU if available, otherwise CPU)
language="en"

from multiprocessing import Queue

# GPU
if torch.cuda.is_available():
  device = "cuda"
  compute_type = "float32"
  batch_size = 16
  model_whisperx = "base"

  print_info(f"""Generating SRT File with {device}...""")
else:
  device = "cpu"
  compute_type = "int8"
  batch_size = 1
  model_whisperx = "tiny"

  queue = Queue(maxsize=200)

  print_info(f"""WARNING: Generating SRT File with {device}...""")


# Model WhisperX
model = whisperx.load_model(model_whisperx, device=device, language=language, compute_type=compute_type) # Choose "base" or "large" model

# Transcribe audio
aligned_segments = model.transcribe(audio_output, batch_size=batch_size)

# Align with forced alignment
alignment_model, metadata = whisperx.load_align_model(language_code=aligned_segments["language"], device=device)
aligned_segments = whisperx.align(aligned_segments["segments"], alignment_model, metadata, audio_output, device)

if not experiment_mode:
  # Generate SRT file with aligned sentences
  with open(subtitles_output, "w") as f:
      for i, segment in enumerate(aligned_segments["segments"], 1):
          # Get start and end times in SRT format
          start_time = seconds_to_srt_timestamp(segment["start"])
          end_time = seconds_to_srt_timestamp(segment["end"])

          # Write SRT entry
          f.write(f"{i}\n{start_time} --> {end_time}\n{segment['text']}\n\n")

  print_info("SRT file generated", subtitles_output)
else:
  # Generate SRT content as a string
  srt_content = ""
  for i, segment in enumerate(aligned_segments["segments"], 1):
      # Get start and end times in SRT format
      start_time = seconds_to_srt_timestamp(segment["start"])
      end_time = seconds_to_srt_timestamp(segment["end"])

      # Append SRT entry to the string
      srt_content += f"{i}\n{start_time} --> {end_time}\n{segment['text']}\n\n"

  # Parse the SRT content directly
  subtitles = list(srt.parse(srt_content))


=== Generating SRT File with cuda...



Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/145M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.31k [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../usr/local/lib/python3.11/dist-packages/whisperx/assets/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cu124. Bad things might happen unless you revert torch to 1.x.


It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(



# Preprocessing - Text

## Text - Load SRT File

In [16]:
if not experiment_mode:
  # Subtitles:
  with open(subtitles_output, "r", encoding="utf-8") as f:
      subtitles = list(srt.parse(f.read()))

## Text - Sentence Segmentation

In [17]:
def format_timedelta(timedelta_obj):
    """Formats a datetime.timedelta object into HH:MM:SS.mmm timestamp.

    Args:
        timedelta_obj: The datetime.timedelta object.

    Returns:
        A string representing the timestamp in HH:MM:SS.mmm format.
    """
    total_seconds = timedelta_obj.total_seconds()
    hours = int(total_seconds // 3600)  # Get hours
    minutes = int((total_seconds % 3600) // 60)  # Get minutes
    seconds = int(total_seconds % 60)  # Get seconds
    milliseconds = int((total_seconds % 1) * 1000)  # Get milliseconds

    return f"{hours:02d}:{minutes:02d}:{seconds:02d}.{milliseconds:03d}"

In [18]:
sentences = []
for i, segment in enumerate(subtitles):
    sentences.append({
        'base_idx': i,
        'start_time': format_timedelta(segment.start),
        'end_time': format_timedelta(segment.end),
        'sentence': segment.content
    })

df_sentences = pd.DataFrame(sentences, columns=['base_idx', 'start_time', 'end_time', 'sentence'])
sentences = df_sentences['sentence'].tolist()
notebook_mode_print(df_sentences)

,base_idx,start_time,end_time,sentence
0,0,00:00:01.274,00:00:08.685,"Hello, this is Lino Cordia and today I want t..."
1,1,00:00:11.057,00:00:13.319,So why is teamwork in the classroom so import...
2,2,00:00:13.359,00:00:17.480,"Well, for our students, it allows them to deve..."
3,3,00:00:17.699,00:00:20.059,"Communication skills, leadership skills, et ce..."
4,4,00:00:20.620,00:00:23.861,"Also, when you're working with a team, you get..."
...,...,...,...,...
69,69,00:05:46.624,00:05:48.245,"And yeah, please reach out."
70,70,00:05:48.384,00:05:49.406,I would love to hear from you.
71,71,00:05:49.526,00:05:50.266,Thank you so much.
72,72,00:05:50.547,00:05:51.487,I am Lino Cordia.


## Text - Paragraph
combination of all subtitle parts.  

WhisperAI enhances transcription with basic punctuation.

In [19]:
paragraph = reduce(lambda acc, seg: acc + seg.strip() + ' ', sentences, '')
parahraph_wordcount = count_words_without_punctuation(paragraph)

# Print the paragraph
notebook_mode_print(paragraph)
print(f"paragraph word count: {parahraph_wordcount}")
print_info("paragraph sample", paragraph)

Hello, this is Lino Cordia and today I want to talk to you about a very important topic, challenging topic, teamwork in the classroom. So why is teamwork in the classroom so important? Well, for our students, it allows them to develop a bunch of new skills, right? Communication skills, leadership skills, et cetera. Also, when you're working with a team, you get different perspectives. Ideally, you are part of a team that has people with different genders, people with different age groups, people with different academic backgrounds, right? So when you're talking to them, you get all these fresh perspectives that inform your the task that you're trying to solve. Also teams will motivate you, they will support you, you will feel empowered by them ideally, right? This is like the things that should happen. And also this is how the world works, right? Pretty much everything we do, we need to do it as a team. Now here's the thing about teamwork in the classroom. Students hate it. But why do 

## Text - Paragraph Summarized

##### Time Taken: ~1min

In [20]:
print_info("Summarizing Paragraph")


=== Summarizing Paragraph



In [21]:
paragraph_summarized = ""

if parahraph_wordcount > 0:
  # Model: Longformer Encoder-Decoder
  model_name = "allenai/led-base-16384"
  tokenizer = LEDTokenizer.from_pretrained(model_name)
  model = LEDForConditionalGeneration.from_pretrained(model_name)
  text = paragraph

  # Tokenization
  inputs = tokenizer(text, return_tensors="pt", max_length=16384, truncation=True)

  # Calculate dynamic summary length
  summary_length_percentage = hyperparameters["auto_summary"]["summary_length_percentage"]
  min_summary_length = hyperparameters["auto_summary"]["min_summary_length"]
  max_summary_length = hyperparameters["auto_summary"]["max_summary_length"]


  input_length = len(inputs["input_ids"][0])
  summary_length = int(input_length * summary_length_percentage)
  summary_length = max(min_summary_length, min(summary_length, max_summary_length))

  # Summary Generation
  summary_ids = model.generate(
      inputs["input_ids"],
      max_length=summary_length,
      min_length=min_summary_length,
      length_penalty=1.2,
      num_beams=4,
      early_stopping=True
  )

  paragraph_summarized = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  print_info("paragraph summarized", paragraph_summarized)

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/648M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/648M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Input ids are automatically padded from 1141 to 2048 to be a multiple of `config.attention_window`: 1024



=== paragraph summarized

--- Preview:
Hello, this is Lino Cordia and today I want to talk to you about a very importan...


In [22]:
# Simple Metrics
print_info("Simple Metrics")

original_length = len(paragraph)
summary_length = len(paragraph_summarized)

print(f"original length: {original_length}")
print(f"summary length: {summary_length}")

if original_length > 0 and original_length > summary_length:
  summarization_ratio = (original_length - summary_length) / original_length
  print(f"Summarized/Original Length Ratio: {summarization_ratio:.2f}")
elif original_length == 0:
  print_info("WARN: 0 Words found in video")


=== Simple Metrics

original length: 5209
summary length: 1555
Summarized/Original Length Ratio: 0.70


# Text

## Metric 1: Sentence- Summarized Paragraph Relevancy (Cosine Similarity)

##### Time Taken: ~2min

In [23]:
print_section("Metric 1: Sentence-Summarized Relevancy")

 Metric 1: Sentence-Summarized Relevancy



In [24]:
if parahraph_wordcount < hyperparameters["metric_1"]["min_words"]:
  skip_text_metrics = True
  print_info(f"Warning: Video contained only {parahraph_wordcount} words, \
  which did not meet minimum wordcount threshold to use this metric. Skipping.")

else:
  skip_text_metrics = False

In [25]:
if not skip_text_metrics:
  # Model Setup
  model_instructor_xl = SentenceTransformer(
    'hkunlp/instructor-xl',
    device='cuda' if torch.cuda.is_available() else 'cpu'
  )

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [26]:
if not skip_text_metrics:
  instruction = "Represent the text for semantic similarity"

  # 3: Embedding (with Tokenization)
  paragraph_embedding = model_instructor_xl.encode(
    [(instruction, paragraph_summarized)],
    convert_to_tensor=True,
    max_length=4096,
    truncation=True
  )

  sentence_embeddings = model_instructor_xl.encode(
    [(instruction, s) for s in sentences],
    convert_to_tensor=True,
    max_length=4096,
    truncation=True
  )

Embedding Explanation  
The [CLS] (classification) token is often used in transformer models to represent the overall meaning or summary of the input sequence. By extracting its embedding, you're essentially obtaining a representation that captures the main point or essence of the paragraph.

In [27]:
if not skip_text_metrics:
  # 4: Relevance scores
  relevance_scores = [
    torch.cosine_similarity(paragraph_embedding, sentence_embedding).item()
    for sentence_embedding in sentence_embeddings
  ]

  # Normalization: min-max normalization
  min_score = min(relevance_scores)
  max_score = max(relevance_scores)
  normalized_scores = [(score - min_score) / (max_score - min_score) for score in relevance_scores]

  # round
  normalized_scores = [np.format_float_positional(score, precision=2, unique=False, fractional=False, trim='k') for score in normalized_scores]

In [28]:
# 5: Display Results
drop_if_exists(df_sentences, "metric_1_score")

if not skip_text_metrics:
  df_sentences.insert(0, "metric_1_score", normalized_scores)
else:
  df_sentences.insert(1, "metric_1_score", 0)

notebook_mode_print(df_sentences)

,metric_1_score,base_idx,start_time,end_time,sentence
0,0.98,0,00:00:01.274,00:00:08.685,"Hello, this is Lino Cordia and today I want t..."
1,1.0,1,00:00:11.057,00:00:13.319,So why is teamwork in the classroom so import...
2,0.63,2,00:00:13.359,00:00:17.480,"Well, for our students, it allows them to deve..."
3,0.53,3,00:00:17.699,00:00:20.059,"Communication skills, leadership skills, et ce..."
4,0.67,4,00:00:20.620,00:00:23.861,"Also, when you're working with a team, you get..."
...,...,...,...,...,...
69,0.15,69,00:05:46.624,00:05:48.245,"And yeah, please reach out."
70,0.18,70,00:05:48.384,00:05:49.406,I would love to hear from you.
71,0.041,71,00:05:49.526,00:05:50.266,Thank you so much.
72,0.20,72,00:05:50.547,00:05:51.487,I am Lino Cordia.


In [29]:
# Interactive Sheet for easy exporting
# from google.colab import sheets
# sheet = sheets.InteractiveSheet(df=df_sentences)

# Video

## Metric 2: Shot Detection

##### Time taken ~1min

In [30]:
# 1: Detect Scenes

scene_list = detect(video_input,
                    ContentDetector(
                        threshold=hyperparameters["metric_2"]["threshold"],
                        min_scene_len=hyperparameters["metric_2"]["min_scene_len"]
                    ))

notebook_mode_print(scene_list)

INFO:pyscenedetect:Detecting scenes...


[(00:00:00.000 [frame=0, fps=30.300], 00:00:09.637 [frame=292, fps=30.300]), (00:00:09.637 [frame=292, fps=30.300], 00:00:57.854 [frame=1753, fps=30.300]), (00:00:57.854 [frame=1753, fps=30.300], 00:01:04.455 [frame=1953, fps=30.300]), (00:01:04.455 [frame=1953, fps=30.300], 00:02:58.448 [frame=5407, fps=30.300]), (00:02:58.448 [frame=5407, fps=30.300], 00:03:36.632 [frame=6564, fps=30.300]), (00:03:36.632 [frame=6564, fps=30.300], 00:05:27.160 [frame=9913, fps=30.300]), (00:05:27.160 [frame=9913, fps=30.300], 00:05:53.628 [frame=10715, fps=30.300])]


In [31]:
# 2: Segment Scoring
scene_segments = [(end.get_seconds() - start.get_seconds(), start.get_timecode(), end.get_timecode())
                  for start, end in scene_list]

# Convert to DataFrame
df_scenes = pd.DataFrame(scene_segments, columns=["duration", "start_time", "end_time"])

total_duration = df_scenes["duration"].sum()

# Calculate score: row's duration / total duration - this is Normalization
df_scenes["normalized_score"] = df_scenes["duration"] / total_duration
df_scenes["normalized_score"] = df_scenes["normalized_score"].round(2)

# Min-Max scaling
df_scenes["score"] = \
 (df_scenes["normalized_score"] - df_scenes["normalized_score"].min()) / \
(df_scenes["normalized_score"].max() - df_scenes["normalized_score"].min())
df_scenes["score"] = df_scenes["score"].round(2)

notebook_mode_print(df_scenes)

,duration,start_time,end_time,normalized_score,score
0,9.636908,00:00:00.000,00:00:09.637,0.03,0.03
1,48.217545,00:00:09.637,00:00:57.854,0.14,0.40
2,6.600622,00:00:57.854,00:01:04.455,0.02,0.00
3,113.992745,00:01:04.455,00:02:58.448,0.32,1.00
4,38.184599,00:02:58.448,00:03:36.632,0.11,0.30
5,110.527418,00:03:36.632,00:05:27.160,0.31,0.97
6,26.468495,00:05:27.160,00:05:53.628,0.07,0.17


In [32]:
# Function to compute weighted scene score for each sentence
def compute_sentence_score(sentence_row):
    total_score = 0
    scene_number_start = -1
    scene_number_end = -1
    sentence_start = ts_to_s(sentence_row['start_time'])
    sentence_end = ts_to_s(sentence_row['end_time'])
    sentence_duration = sentence_end - sentence_start

    # find scene start and corresponding index
    for scene_idx, scene_row in df_scenes.iterrows():
      scene_score = scene_row['score']
      scene_start = ts_to_s(scene_row['start_time'])
      scene_end = ts_to_s(scene_row['end_time'])
      scene_duration = scene_row['duration']

      # case 1: Fully contained
      if sentence_start >= scene_start and sentence_end <= scene_end:
        scene_number_start = scene_idx
        scene_number_end = scene_idx

        total_score = scene_score
        break

      # case 2: start-contained, end extends
      elif sentence_start >= scene_start and sentence_start < scene_end and sentence_end >= scene_end:
        scene_number_start = scene_idx

        percentage = (scene_end - sentence_start) / sentence_duration
        total_score += scene_score * percentage

      # case 3: end-contained, start extends
      elif sentence_start <= scene_start and sentence_end <= scene_end and sentence_end > scene_start:
        scene_number_end = scene_idx

        percentage = (sentence_end - scene_start) / sentence_duration
        total_score += scene_score * percentage

      # case 4: mid part
      elif sentence_start <= scene_start and sentence_end >= scene_end:
        percentage = scene_duration / sentence_duration
        total_score += scene_score * percentage

    return total_score,scene_number_start,scene_number_end

In [33]:
# 3: Apply score to contained sentence and track scene_number

# Drop existing columns if needed
drop_if_exists(df_sentences, "metric_2_score")
drop_if_exists(df_sentences, "scene_number_start")
drop_if_exists(df_sentences, "scene_number_end")

# Insert columns for metric_2_score and scene_number
df_sentences.insert(1, "metric_2_score", 0)
df_sentences.insert(1, "scene_number_end", 0)
df_sentences.insert(1, "scene_number_start", 0)


if not skip_text_metrics:
  # Apply the function to get the score and scene_number
  df_sentences[['metric_2_score', 'scene_number_start', 'scene_number_end']] = df_sentences.apply(compute_sentence_score, axis=1, result_type='expand')

  df_sentences['scene_number_start'] = df_sentences['scene_number_start'].astype(int)
  df_sentences['scene_number_end'] = df_sentences['scene_number_end'].astype(int)

# Replace sentences_df with scenes
else:
  print_info("Metric 1 Skipped Defaulting to Metric 2-only evaluation...")

  df_scenes_as_sentences = df_scenes.assign(
      base_idx=df_scenes.index,
      scene_number_start=df_scenes.index,
      scene_number_end=df_scenes.index,
      metric_1_score=0,
      metric_2_score=df_scenes.score,
      start_time=df_scenes.start_time,
      end_time=df_scenes.end_time,
      sentence= ""
  )

  df_sentences = df_scenes_as_sentences

  # only select columns from df_sentences
  # df_scenes_as_sentences = df_scenes_as_sentences[df_sentences.columns]
  # df_sentences = pd.concat([df_sentences, df_scenes_as_sentences], ignore_index=True)

# Display the updated DataFrame
notebook_mode_print(df_sentences)

,metric_1_score,scene_number_start,scene_number_end,metric_2_score,base_idx,start_time,end_time,sentence
0,0.98,0,0,0.03,0,00:00:01.274,00:00:08.685,"Hello, this is Lino Cordia and today I want t..."
1,1.0,1,1,0.40,1,00:00:11.057,00:00:13.319,So why is teamwork in the classroom so import...
2,0.63,1,1,0.40,2,00:00:13.359,00:00:17.480,"Well, for our students, it allows them to deve..."
3,0.53,1,1,0.40,3,00:00:17.699,00:00:20.059,"Communication skills, leadership skills, et ce..."
4,0.67,1,1,0.40,4,00:00:20.620,00:00:23.861,"Also, when you're working with a team, you get..."
...,...,...,...,...,...,...,...,...
69,0.15,6,6,0.17,69,00:05:46.624,00:05:48.245,"And yeah, please reach out."
70,0.18,6,6,0.17,70,00:05:48.384,00:05:49.406,I would love to hear from you.
71,0.041,6,6,0.17,71,00:05:49.526,00:05:50.266,Thank you so much.
72,0.20,6,6,0.17,72,00:05:50.547,00:05:51.487,I am Lino Cordia.


# Final Score - Metric Weighting

In [34]:
def smooth_consecutive_zeros(arr, discount=0.8):
    arr = np.array(arr, dtype=float)
    i = 0
    while i < len(arr):
        if arr[i] == 0:
            start = i
            while i < len(arr) and arr[i] == 0:
                i += 1
            end = i

            prev = next((arr[j] for j in range(start-1, -1, -1) if arr[j] != 0), None)
            next_ = next((arr[j] for j in range(end, len(arr)) if arr[j] != 0), None)

            if prev is not None and next_ is not None:
                base_val = ((prev + next_) / 2) * discount
            elif prev is not None:
                base_val = prev * discount
            elif next_ is not None:
                base_val = next_ * discount
            else:
                base_val = 0  # fallback if all values are 0

            arr[start:end] = base_val
        else:
            i += 1
    return arr.tolist()

In [35]:
def insert_silence_rows(df):
    if len(df) < 1:
        return df.copy()

    new_rows = []
    base_idx_counter = 0

    # Check for silence at the beginning (if first start_time != "00:00:00.000")
    first_row = df.iloc[0]
    if first_row['start_time'] != "00:00:00.000":
        silence_row = {
            'metric_final': 0.0,
            'start_time': "00:00:00.000",
            'end_time': first_row['start_time'],
            'base_idx': -1,
            'sentence': ''
        }
        new_rows.append(pd.Series(silence_row))



    # Process all rows
    for i in range(len(df)):
        # Add current row (with updated base_idx)
        curr_row = df.iloc[i].copy()
        curr_row['base_idx'] = base_idx_counter
        new_rows.append(curr_row)
        base_idx_counter += 1

        # Check for silence between current and next row
        if i < len(df) - 1:
            next_row = df.iloc[i + 1]
            if curr_row['end_time'] != next_row['start_time']:
                silence_row = {
                    'metric_final': 0.0,
                    'start_time': curr_row['end_time'],
                    'end_time': next_row['start_time'],
                    'base_idx': -1,
                    'sentence': ''
                }
                new_rows.append(pd.Series(silence_row))



    # Check for silence at the ending (if last start_time < video length)
    video_end_s = get_video_length(video_input)
    video_end_ts = seconds_to_srt_timestamp(video_end_s)

    last_row = df.iloc[-1]
    last_row_seconds = ts_to_s(last_row['end_time'])

    if last_row_seconds < video_end_s:
        silence_row = {
            'metric_final': 0.0,
            'start_time': last_row['end_time'],
            'end_time': video_end_ts,
            'base_idx': -1,
            'sentence': ''
        }
        new_rows.append(pd.Series(silence_row))


    # new_rows to df
    final_df = pd.DataFrame(new_rows).reset_index(drop=True)
    final_df['base_idx'] = final_df.index  # update base_idx to match row order
    return final_df

In [36]:
# Normalized Weigthed average
w1 = hyperparameters['metric_1']['weight'] if not skip_text_metrics else 0
w2 = hyperparameters['metric_2']['weight']
weight_sum = w1 + w2
w1_normalized = w1 / weight_sum
w2_normalized = w2 / weight_sum

# Metric 1 Apply
drop_if_exists(df_sentences, "metric_1_weighted")
df_sentences.insert(0, "metric_1_weighted", 1)
df_sentences['metric_1_score'] = df_sentences['metric_1_score'].astype(float)

df_sentences['metric_1_weighted'] = w1_normalized * df_sentences['metric_1_score']

# Metric 2 Apply
drop_if_exists(df_sentences, "metric_2_weighted")
df_sentences.insert(0, "metric_2_weighted", 1)
df_sentences['metric_2_score'] = df_sentences['metric_2_score'].astype(float)

df_sentences['metric_2_weighted'] = w2_normalized * df_sentences['metric_2_score']


# Metric Final Apply
drop_if_exists(df_sentences, "metric_final")
df_sentences.insert(0, "metric_final", 1)

df_sentences['metric_final'] = df_sentences['metric_1_weighted'] + df_sentences['metric_2_weighted']
df_sentences["metric_final"] = df_sentences["metric_final"].round(2)

# Reorder
df_sentences = df_sentences[[
    'metric_final',
    'metric_1_weighted',
    'metric_1_score',
    'metric_2_weighted',
    'metric_2_score',
    'scene_number_start',
    'scene_number_end',
    'base_idx',
    'start_time',
    'end_time',
    'sentence'
    ]].copy()

notebook_mode_print(df_sentences)

,metric_final,metric_1_weighted,metric_1_score,metric_2_weighted,metric_2_score,scene_number_start,scene_number_end,base_idx,start_time,end_time,sentence
0,0.76,0.753846,0.980,0.006923,0.03,0,0,0,00:00:01.274,00:00:08.685,"Hello, this is Lino Cordia and today I want t..."
1,0.86,0.769231,1.000,0.092308,0.40,1,1,1,00:00:11.057,00:00:13.319,So why is teamwork in the classroom so import...
2,0.58,0.484615,0.630,0.092308,0.40,1,1,2,00:00:13.359,00:00:17.480,"Well, for our students, it allows them to deve..."
3,0.50,0.407692,0.530,0.092308,0.40,1,1,3,00:00:17.699,00:00:20.059,"Communication skills, leadership skills, et ce..."
4,0.61,0.515385,0.670,0.092308,0.40,1,1,4,00:00:20.620,00:00:23.861,"Also, when you're working with a team, you get..."
...,...,...,...,...,...,...,...,...,...,...,...
69,0.15,0.115385,0.150,0.039231,0.17,6,6,69,00:05:46.624,00:05:48.245,"And yeah, please reach out."
70,0.18,0.138462,0.180,0.039231,0.17,6,6,70,00:05:48.384,00:05:49.406,I would love to hear from you.
71,0.07,0.031538,0.041,0.039231,0.17,6,6,71,00:05:49.526,00:05:50.266,Thank you so much.
72,0.19,0.153846,0.200,0.039231,0.17,6,6,72,00:05:50.547,00:05:51.487,I am Lino Cordia.


#### Silence Smoothing

In [37]:
# Smoothing
smoothing_discount = hyperparameters['deletion_metric']['smoothing']

# Insert Silence rows
df_sentences = insert_silence_rows(df_sentences)

# SMOOTH SILENCE SCORES
df_sentences["metric_final"] = smooth_consecutive_zeros(df_sentences["metric_final"].tolist(), discount=smoothing_discount)

notebook_mode_print(df_sentences)

,metric_final,start_time,end_time,base_idx,sentence,metric_1_weighted,metric_1_score,metric_2_weighted,metric_2_score,scene_number_start,scene_number_end
0,0.608,00:00:00.000,00:00:01.274,0,,NaN,NaN,NaN,NaN,NaN,NaN
1,0.760,00:00:01.274,00:00:08.685,1,"Hello, this is Lino Cordia and today I want t...",0.753846,0.98,0.006923,0.03,0.0,0.0
2,0.648,00:00:08.685,00:00:11.057,2,,NaN,NaN,NaN,NaN,NaN,NaN
3,0.860,00:00:11.057,00:00:13.319,3,So why is teamwork in the classroom so import...,0.769231,1.00,0.092308,0.40,1.0,1.0
4,0.576,00:00:13.319,00:00:13.359,4,,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
144,0.104,00:05:50.266,00:05:50.547,144,,NaN,NaN,NaN,NaN,NaN,NaN
145,0.190,00:05:50.547,00:05:51.487,145,I am Lino Cordia.,0.153846,0.20,0.039231,0.17,6.0,6.0
146,0.092,00:05:51.487,00:05:51.706,146,,NaN,NaN,NaN,NaN,NaN,NaN
147,0.040,00:05:51.706,00:05:52.307,147,See you next time.,0.000000,0.00,0.039231,0.17,6.0,6.0


### Deletion Metric

In [38]:
# Threshold
threshold = hyperparameters['deletion_metric']['threshold']

# Percentage
percentile = df_sentences['metric_final'].quantile(threshold)
filtered_df_to_keep = df_sentences[df_sentences['metric_final'] >= percentile]
filtered_df_to_delete = df_sentences[df_sentences['metric_final'] < percentile]

# Timestamps
# sample_timestamps = [('00:00:00.00','00:00:01.25'), ('00:00:08.766', '00:00:11.042')]
sentence_timestamps = list(zip(filtered_df_to_keep['start_time'], filtered_df_to_keep['end_time']))


notebook_mode_print(sentence_timestamps)

[('00:00:00.000', '00:00:01.274'), ('00:00:01.274', '00:00:08.685'), ('00:00:08.685', '00:00:11.057'), ('00:00:11.057', '00:00:13.319'), ('00:00:13.319', '00:00:13.359'), ('00:00:13.359', '00:00:17.480'), ('00:00:17.480', '00:00:17.699'), ('00:00:17.699', '00:00:20.059'), ('00:00:20.059', '00:00:20.620'), ('00:00:20.620', '00:00:23.861'), ('00:00:23.861', '00:00:24.042'), ('00:00:24.042', '00:00:34.804'), ('00:00:34.804', '00:00:34.825'), ('00:00:34.825', '00:00:39.506'), ('00:00:42.225', '00:00:43.408'), ('00:00:43.408', '00:00:49.691'), ('00:00:49.691', '00:00:49.710'), ('00:00:52.551', '00:00:54.213'), ('00:00:54.213', '00:00:54.253'), ('00:00:54.253', '00:00:57.094'), ('00:00:57.094', '00:00:59.396'), ('00:00:59.396', '00:01:01.155'), ('00:01:01.155', '00:01:02.356'), ('00:01:05.367', '00:01:06.507'), ('00:01:06.507', '00:01:06.628'), ('00:01:06.628', '00:01:09.087'), ('00:01:09.087', '00:01:09.769'), ('00:01:09.769', '00:01:13.950'), ('00:01:13.950', '00:01:14.391'), ('00:01:14.39

In [39]:
if not skip_text_metrics:
  threshold = hyperparameters['deletion_metric']['threshold']

  filtered_df = df_sentences.copy()

  # Count words of each sentence
  filtered_df['word_count'] = filtered_df['sentence'].apply(count_words_without_punctuation)

  # Target word count to keep (e.g., 80% of total words when threshold = 20%)
  total_word_count = filtered_df['word_count'].sum()
  target_word_count = round(total_word_count * (1 - threshold))

  # Sort the DataFrame by 'metric_final' (low to high score)
  filtered_df = filtered_df.sort_values(by='metric_final')

  # Initialize the variable for the current word count of remaining sentences
  current_word_count = total_word_count

  # Create a new column for keeping/deleting sentences
  filtered_df["threshold_keep"] = 1  # Default to keeping all sentences

  # Iteratively mark sentences for deletion until the target word count is reached
  for idx, row in filtered_df.iterrows():
      # if current_word_count <= target_word_count: # deleting 1 more
      if current_word_count - row["word_count"]<= target_word_count: # deleting 1 less
          break  # Stop once we've removed enough words

      # Update the current word count after marking the sentence
      current_word_count -= row["word_count"]

      # Mark sentence for deletion
      filtered_df.at[idx, "threshold_keep"] = 0  # 0 = delete, 1 = keep

  # Restore sort
  filtered_df = filtered_df.sort_values(by='base_idx')

  # Text to keep/delete
  filtered_df_to_delete = filtered_df[filtered_df['threshold_keep'] == 0].copy()
  filtered_df_to_keep = filtered_df[filtered_df['threshold_keep'] == 1].copy()

  # Timestamps
  sentence_timestamps = list(zip(filtered_df_to_keep['start_time'], filtered_df_to_keep['end_time']))


  notebook_mode_print(filtered_df)

,metric_final,start_time,end_time,base_idx,sentence,metric_1_weighted,metric_1_score,metric_2_weighted,metric_2_score,scene_number_start,scene_number_end,word_count,threshold_keep
0,0.608,00:00:00.000,00:00:01.274,0,,NaN,NaN,NaN,NaN,NaN,NaN,0,1
1,0.760,00:00:01.274,00:00:08.685,1,"Hello, this is Lino Cordia and today I want t...",0.753846,0.98,0.006923,0.03,0.0,0.0,24,1
2,0.648,00:00:08.685,00:00:11.057,2,,NaN,NaN,NaN,NaN,NaN,NaN,0,1
3,0.860,00:00:11.057,00:00:13.319,3,So why is teamwork in the classroom so import...,0.769231,1.00,0.092308,0.40,1.0,1.0,9,1
4,0.576,00:00:13.319,00:00:13.359,4,,NaN,NaN,NaN,NaN,NaN,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,0.104,00:05:50.266,00:05:50.547,144,,NaN,NaN,NaN,NaN,NaN,NaN,0,0
145,0.190,00:05:50.547,00:05:51.487,145,I am Lino Cordia.,0.153846,0.20,0.039231,0.17,6.0,6.0,4,0
146,0.092,00:05:51.487,00:05:51.706,146,,NaN,NaN,NaN,NaN,NaN,NaN,0,0
147,0.040,00:05:51.706,00:05:52.307,147,See you next time.,0.000000,0.00,0.039231,0.17,6.0,6.0,4,0


### Text to Keep

In [40]:
notebook_mode_print(filtered_df_to_keep[['metric_final', 'start_time', 'end_time', 'sentence']])

,metric_final,start_time,end_time,sentence
0,0.608,00:00:00.000,00:00:01.274,
1,0.760,00:00:01.274,00:00:08.685,"Hello, this is Lino Cordia and today I want t..."
2,0.648,00:00:08.685,00:00:11.057,
3,0.860,00:00:11.057,00:00:13.319,So why is teamwork in the classroom so import...
4,0.576,00:00:13.319,00:00:13.359,
...,...,...,...,...
124,0.416,00:05:14.670,00:05:14.891,
125,0.490,00:05:14.891,00:05:19.716,I find that it's easy to identify what each me...
127,0.510,00:05:20.497,00:05:26.403,It's easy also for them for communication purp...
135,0.410,00:05:33.874,00:05:37.975,"I'm just sharing what I've done, what has work..."


In [41]:
text_to_keep = " ".join(filtered_df_to_keep['sentence'].tolist())
paragraph_trimmed = text_to_keep

notebook_mode_print(text_to_keep)

  Hello, this is Lino Cordia and today I want to talk to you about a very important topic, challenging topic, teamwork in the classroom.   So why is teamwork in the classroom so important?  Well, for our students, it allows them to develop a bunch of new skills, right?  Communication skills, leadership skills, et cetera.  Also, when you're working with a team, you get different perspectives.  Ideally, you are part of a team that has people with different genders, people with different age groups, people with different academic backgrounds, right? So when you're talking to them, you get all these fresh perspectives that inform your Also teams will motivate you, they will support you, you will feel empowered by them ideally, right? Pretty much everything we do, we need to do it as a team.  Now here's the thing about teamwork in the classroom.  But why do they hate it? They don't know their teammates or they do know their teammates, but they don't like them.  They're also concerned about 

### Text to Delete

In [42]:
notebook_mode_print(filtered_df_to_delete[['metric_final', 'start_time', 'end_time', 'sentence']])

,metric_final,start_time,end_time,sentence
12,0.400,00:00:34.804,00:00:34.825,
14,0.296,00:00:39.506,00:00:40.426,
15,0.280,00:00:40.426,00:00:42.225,the task that you're trying to solve.
16,0.368,00:00:42.225,00:00:43.408,
18,0.376,00:00:49.691,00:00:49.710,
...,...,...,...,...
144,0.104,00:05:50.266,00:05:50.547,
145,0.190,00:05:50.547,00:05:51.487,I am Lino Cordia.
146,0.092,00:05:51.487,00:05:51.706,
147,0.040,00:05:51.706,00:05:52.307,See you next time.


In [43]:
text_to_delete = " ".join(filtered_df_to_delete['sentence'].tolist())

notebook_mode_print(text_to_delete)

   the task that you're trying to solve.   This is like the things that should happen.  And also this is how the world works, right?   Students hate it.   Well, these are the things that I can identify.    They're concerned because   A lot of it is out of their control.    These are the things that I do.  Something that doesn't really impact their grade or if it does, it's tiny, it's minimal.  And after a few weeks, I saw that this team was working really well together.   And their answer was we had dinner together.   Go and grab a coffee, go and get lunch together, something like that is going to help you connect.   That works. You need to determine when is a good idea for self-selection or when is a good idea to think of.   This is what I do.   There's tables with four chairs, right?     But what are your strategies?  I would love to learn.  I'm not an expert in team building, right?    And yeah, please reach out.  I would love to hear from you.  Thank you so much.  I am Lino Cordia.

In [44]:
if not skip_text_metrics:
  words_kept = filtered_df_to_keep['word_count'].sum()
  words_delete = filtered_df_to_delete['word_count'].sum()

  notebook_mode_print(f"Text deletion: {round(words_delete/(words_kept+words_delete) * 100, 2)}%")
  notebook_mode_print(f"Sentence deletion: {round(len(filtered_df_to_delete) / (len(filtered_df_to_delete) + len(filtered_df_to_keep)) * 100, 2)}%")

Text deletion: 19.64%
Sentence deletion: 43.62%


# PostProcessing

#### Time Taken: ~1.5min
6min video: ~2min to process, ~30sec to download

In [45]:
print_section("Postprocessing")

============ Postprocessing ============



In [46]:
def skim_video(input_video, output_video, segments_to_retain):
    """
    Skims a video by keeping only the specified segments and removes others.

    Args:
        input_video (str): Path to the input video file.
        output_video (str): Path to the output video file.
        segments_to_retain (list of tuples): List of tuples where each tuple contains
                                             (start_time, end_time) in seconds to retain.
    """
    print_info("Processed video...")

    # Prepare the select filter for video (only select the specified ranges)
    video_select_filter = '+'.join([
        f"between(t,{start},{end})"
        for start, end in segments_to_retain
    ])

    # Prepare the select filter for audio (only select the specified ranges)
    audio_select_filter = '+'.join([
        f"between(t,{start},{end})"
        for start, end in segments_to_retain
    ])

    # Construct the ffmpeg command with the specified filters
    ffmpeg_command = [
        "ffmpeg",
        "-y",
        "-i", input_video,
        "-vf", f"select='{video_select_filter}',setpts=N/FRAME_RATE/TB",
        "-af", f"aselect='{audio_select_filter}',asetpts=N/SR/TB",
        "-threads", str(os.cpu_count()),
         "-preset", "ultrafast",
        output_video
    ]

    # Run the FFmpeg command and capture the output
    result = subprocess.run(ffmpeg_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Check if FFmpeg finished successfully or if there were errors
    if result.returncode != 0:
        print("FFmpeg Error:")
        print(result.stderr.decode())  # Print the error output
    else:
        print_info("Video processed successfully.")

In [47]:
def generate_keep_timestamps(timestamps_to_remove, video_length=None):
    """
    Given a list of timestamps to remove from a video, generates the list of timestamps to keep.

    Args:
        timestamps_to_remove (list of tuples): List of segments to remove (start_time, end_time) in seconds.
        video_length (float, optional): Total length of the video in seconds. If not provided, the last segment's end is used.

    Returns:
        list of tuples: Segments to keep.
    """
    # Sort the timestamps to remove by their start times (just in case they're out of order)
    timestamps_to_remove.sort()

    # Initialize the list of segments to keep
    timestamps = []

    # If the first removal starts after 0, keep from the start of the video to the first removal
    if timestamps_to_remove[0][0] > 0:
        timestamps.append((0.0, timestamps_to_remove[0][0]))

    # Now, for each consecutive pair of timestamps to remove, keep the time between them
    for i in range(len(timestamps_to_remove) - 1):
        end_of_previous_removal = timestamps_to_remove[i][1]
        start_of_next_removal = timestamps_to_remove[i + 1][0]

        # If there's a gap, keep that gap
        if end_of_previous_removal < start_of_next_removal:
            timestamps.append((end_of_previous_removal, start_of_next_removal))

    # If there is time left after the last removal, keep it
    if video_length is not None:
        last_end_time = timestamps_to_remove[-1][1]
        if last_end_time < video_length:
            timestamps.append((last_end_time, video_length))

    return timestamps

In [48]:
# Timestamp pre-processing
original_video_length = get_video_length(video_input)

# Dev mode: export shorter video
video_length = min(original_video_length, video_export_max_length_seconds) if video_export_max_length_seconds > 0 else original_video_length

# Trim Method 1: Video with sentences to remove, removed
# timestamps_to_remove = list(map(lambda x: (ts_to_s(x[0]), ts_to_s(x[1])), sentence_timestamps))
# timestamps_to_keep = generate_keep_timestamps(timestamps_to_remove, video_length)

# Trim method 2: Video of only sentences to keep
timestamps_to_keep = list(map(lambda x: (ts_to_s(x[0]), ts_to_s(x[1])), sentence_timestamps))

# print(f"Timestamps to remove: {timestamps_to_remove}")
notebook_mode_print(f"Timestamps to keep: {timestamps_to_keep}")

Timestamps to keep: [(0.0, 1.274), (1.274, 8.685), (8.685, 11.057), (11.057, 13.319), (13.319, 13.359), (13.359, 17.48), (17.48, 17.699), (17.699, 20.059), (20.059, 20.62), (20.62, 23.861), (23.861, 24.042), (24.042, 34.804), (34.825, 39.506), (43.408, 49.691), (54.253, 57.094), (57.094, 59.396), (59.396, 61.155), (65.367, 66.507), (69.769, 73.95), (73.95, 74.391), (74.391, 77.111), (77.111, 77.152), (77.152, 84.495), (84.775, 86.215), (90.138, 96.12), (96.12, 96.322), (96.322, 99.305), (100.926, 102.766), (104.629, 104.709), (104.709, 108.331), (108.331, 108.652), (108.652, 110.953), (112.013, 113.795), (113.795, 113.834), (113.834, 117.018), (117.018, 118.367), (118.367, 124.552), (124.552, 124.611), (124.611, 133.9), (133.9, 134.479), (134.479, 137.703), (137.703, 137.861), (137.861, 151.376), (151.376, 152.137), (152.137, 159.818), (159.818, 160.038), (164.878, 165.9), (165.9, 170.02), (170.02, 171.181), (171.181, 177.52), (177.52, 180.508), (180.508, 182.55), (182.55, 183.25), (18

In [49]:
# Skim Video
if not experiment_mode:
  skim_video(video_input, video_output_skimmed, timestamps_to_keep)


=== Processed video...


=== Video processed successfully.



In [50]:
# Download
if not experiment_mode:
  if notebook_mode:
    print_info("Downloading video...")
    files.download(video_output_skimmed)


=== Downloading video...



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [51]:
# Export original text
if export_original_text:
  paragraph_to_file(paragraph, filename_paragraph_original)

  if notebook_mode:
    files.download(filename_paragraph_original)

# Export trimmed text
if export_trimmed_text:
  paragraph_to_file(paragraph_trimmed, filename_paragraph_trimmed)

  if notebook_mode:
    files.download(filename_paragraph_trimmed)

# Export summarized text
if export_summarized_text:
  paragraph_to_file(paragraph_summarized, filename_paragraph_summarized)

  if notebook_mode:
    files.download(filename_paragraph_summarized)

In [52]:
# Simple Metrics
if not experiment_mode:
  print_section("Simple Metrics")

  original_video_length = get_video_length(video_input)
  print(f"Original Video Length: {original_video_length:.2f}s\n")

  skimmed_video_length = get_video_length(video_output_skimmed)
  print(f"Skimmed Video Length: {skimmed_video_length:.2f}s\n")

  summarization_ratio = (original_video_length - skimmed_video_length) / original_video_length
  print(f"Skimmed/Original Video Length Ratio: {summarization_ratio:.2f}")

============ Simple Metrics ============

Original Video Length: 353.63s

Skimmed Video Length: 278.48s

Skimmed/Original Video Length Ratio: 0.21


### Experiment Export

In [53]:
if experiment_mode:
  df_sentences[['metric_final', 'start_time', 'end_time', 'base_idx', 'sentence']].to_csv(filename_experiment_sentences, index=False)

  with open(filename_experiment_hyperparameters, "w") as f:
    json.dump(hyperparameters, f, indent=2)

  print_info("Experiment files exported.")